In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
# importing dependencies
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import tqdm ##
from tqdm import * ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Take Training Data

In [2]:
folder = 'data/10fold_stacking/' #共同predict對stacling verified data的結果

acc_df = pd.read_csv('data/ens_unverified/validation_ACC_P1S1.csv') #accuracy csv
acc_df.columns = ['model','csv_name','acc']
acc_df = acc_df.filter(['csv_name','acc'])
acc_df['csv_name'] = acc_df['csv_name'].str.replace('_unverified_','_')
files = os.listdir(folder)

ratio_all=0

df_dict = {}
for i,csv in enumerate(files):
    df_name = csv[:csv.rfind('_')]
#     print(df_name)
    if csv.startswith('validation_ACC'):
        continue
    
    ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
    ratio_all += ratio
    if not (df_name in df_dict.keys()):
        df_dict[df_name] = pd.read_csv(os.path.join(folder,csv))
        df1_ = df_dict[df_name].drop('fname',axis=1)
        df1_ = np.array(df1_) * ratio
        df1_ = pd.DataFrame(df1_)
        df1_['acc'] = ratio
#         print(len(df1_))
        df_dict[df_name] = df_dict[df_name].filter(['fname'])
        df_dict[df_name] = pd.merge(df_dict[df_name],df1_,how='inner',right_index=True,left_index=True)
    else:
        df1_name = pd.read_csv(os.path.join(folder,csv))
        df1_ = df1_name.drop('fname',axis=1)
        df1_ = np.array(df1_) * ratio
        df1_ = pd.DataFrame(df1_)
        df1_['acc'] = ratio
#         print(len(df1_))
        df1_name = df1_name.filter(['fname'])
        df1_ = pd.merge(df1_name, df1_,how='inner',right_index=True,left_index=True)
        df_dict[df_name] = df_dict[df_name].append(df1_, ignore_index=True)
#         print(len(df_dict[df_name]))
#     elif csv.startswith('mike_resnet'):
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         if a==0:
#             df1 = pd.read_csv(os.path.join(folder,csv))
#             df1_ = df1.drop('fname',axis=1)
#             df1_ = np.array(df1_) * ratio
#             df1_ = pd.DataFrame(df1_)
#             df1 = df1.filter(['fname'])
#             df1 = pd.merge(df1,df1_,how='inner',right_index=True,left_index=True)
#             a+=1
#         else:
#             df1_name = pd.read_csv(os.path.join(folder,csv))
#             df1_ = df1_name.drop('fname',axis=1)
#             df1_ = np.array(df1_) * ratio
#             df1_ = pd.DataFrame(df1_)
#             df1_name = df1_name.filter(['fname'])
#             df1_ = pd.merge(df1_name, df1_,how='inner',right_index=True,left_index=True)
#             df1.append(df1_, ignore_index=True)
#     elif csv.startswith('mike_cnn2d'):
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         if b==0:
#             df2 = pd.read_csv(os.path.join(folder,csv))
#             df2_ = df2.drop('fname',axis=1)
#             df2_ = np.array(df1_) * ratio
#             df2_ = pd.DataFrame(df1_)
#             df2 = df2.filter(['fname'])
#             df2 = pd.merge(df2,df1_,how='inner',right_index=True,left_index=True)
#             b+=1
#         else:
#             df2.append(pd.read_csv(os.path.join(folder,csv)), ignore_index=True)     
#     elif csv.startswith('mow_cnn2d'):
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         if c==0:
#             df3 = pd.read_csv(os.path.join(folder,csv))
#             c+=1
#         else:
#             df3.append(pd.read_csv(os.path.join(folder,csv)), ignore_index=True)     
    
#     else:
#         print('unknown csv')
#         break
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         print(ratio)
#         ratio_all += ratio
#     df = pd.read_csv(os.path.join(folder,csv),header=None)

for k,v in df_dict.items():
    df_dict[k] = v.sort_values('fname')

_ = list(df_dict.keys())
sum_ = np.zeros((len(df_dict[_[0]]), 42))
for k,v in df_dict.items():
    sum_ += v[v.columns[1:]].values
ratio_ = np.tile(sum_[:, -1], (41, 1)).T
train_X = sum_[:, :-1] / ratio_
#     df_final = pd.DataFrame(sum_)
# df_final

#     sum_ /= ratio_all


# df = pd.merge(df1,df2,on='fname',how='inner')
# df = pd.merge(df,df3,on='fname',how='inner')
# # if df.iloc[0,0] == 'fname':
# df = df.drop('fname',axis=1)
# # df = df.drop(0,axis=1)

# df

# if i==0:
#     train_X = df.values*ratio
# else:
#     train_X += df.values*ratio
train_X.shape

(3710, 41)

## Take Label

In [3]:
label = pd.read_csv('data/train_label.csv',names=['fname','label','verified'],header=0)
dicts_ = pickle.load(open('data/map.pkl','rb'))
label['trans']=label['label'].map(dicts_)
# label = label.drop(['ID','fname','verified','label'],axis=1)
label = pd.merge(label,df_dict[k],on='fname',how='inner')
label = label['trans'].values
print('label like:',label,'data#:', len(label))

label like: [ 1  3  4 ... 14  9 17] data#: 3710


## Split Data to eval

In [4]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

In [5]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [110]:
X_train, Y_train, X_valid, Y_valid = split_valid_set(train_X, label, 0.95)
print(X_train.shape , Y_train.shape)

(3339, 41) (3339,)


## XGB BO

In [8]:
def train_xgb(max_delta_step,reg_lambda,reg_alpha,n_estimators,max_depth, subsample, min_child_weight, gamma, colsample_bytree, colsample_bylevel):
#, learning_rate
# def train_xgb(max_delta_step,reg_lambda,n_estimators,max_depth,min_child_weight,gamma):
    xgb_params = {
#         'tree_method':'gpu_hist',
#         'predictor':'gpu_predictor',
        'nthread': 12,
        'objective': 'multi:softmax',
        'num_class':41,
#         'n_estimators': int(750.6),
#         'eta': 0.01,
        'silent': 1,
#         'seed': 0,
        # for _train_internal
        'eval_metric': ['merror'],
        ######################
        'max_delta_step':int(max_delta_step),
        'reg_lambda': max(reg_lambda, 0),
        'reg_alpha': max(reg_alpha, 0),
        'n_estimators':int(n_estimators),
        'max_depth': int(max_depth),
        'subsample': max(min(subsample, 1), 0),
        'min_child_weight': int(min_child_weight),
#         'alpha': max(alpha, 0),
        'gamma': max(gamma, 0),
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'colsample_bylevel': max(min(colsample_bylevel, 1), 0),
#         'learning_rate': max(min(learning_rate, 1), 0),
    }
    # print(dTrain)
    score = xgb.cv(
        xgb_params, 
        dTrain, 
        num_boost_round=1500, #2000
        early_stopping_rounds=10, #100
        verbose_eval=False,
        maximize=False, #針對eval_metric
        nfold=3, #10
        seed=np.random.get_state()[1][0]
    )
    # logger.info(score)
    return -score['test-merror-mean'].iloc[-1] #回傳給BO去maximize的，只能改中間的值

In [11]:
def train():
#     X, Y, testX, testFid = prepare_data()
#     X, Y, testX = str2float(X, Y, testX)
#     X, testX = scale_data(X, testX)
    
#     if args.select_feature:
        # select feature
#         pass
    for iter in range(30): #the number of trying sampling，挑選幾個XGB存起來??
        np.random.seed(iter)
        logger.info('Iteration %2d, Current random seed: %2d' % (iter, np.random.get_state()[1][0]))
        # sampling data
        # trainX, trainY, validX, validY = sample_data(X, Y)
        trainX = train_X#pca_all #train_X#X_train
        trainY = label#label#train_Y#Y_train
        testX = train_X#pca_all#train_X#X_valid
        testY = label#label#train_Y#Y_valid
        global dTrain
        dTrain = xgb.DMatrix(trainX, label=trainY, nthread=12)
        global dTest
        dTest = xgb.DMatrix(testX, label=testY,nthread=12)

        # bayes_opt selection
        # parameter to be learning 
        logger.info('Setting parameters for BayesianOptimaization')       
        params = {
            'max_delta_step':(0,10),#(0,10), #0~10 0=>nolimit
            'reg_lambda': (70,85), #1e-5,200 #1: 25,60
            'reg_alpha': (0,0.3), #100 1=>0~1
            'n_estimators':(1120,1135), #50~1600 #250~1200 #1: 365,410
            'max_depth': (1,1),#(1, 10), #65 #不能0=>no limit #1:1,16
            'subsample': (0.9, 1), #0~1
            'min_child_weight': (8, 11), #30 1=>0~1
#             'alpha': (0, 10), #等同reg_alpha
            'gamma': (0,1), #30 1=>0~1
            'colsample_bytree': (0.8, 1), #1e-5~1
            'colsample_bylevel': (0.9, 1), #(1e-5,1)
#             'learning_rate': (0.01, 1),
        }
        logger.info('Running BayesianOptimization')
        xgb_bayesopt = BayesianOptimization(train_xgb, params)
        xgb_bayesopt.maximize(init_points=6, n_iter=100) #10,50 #5 ,25
        
        # get the best param
        best_params = xgb_bayesopt.res['max']['max_params']
        logger.info('Iteration: %d, XGBoost max auc: %f' % (iter, xgb_bayesopt.res['max']['max_val']))
        for param, val in best_params.items():
            logger.info('Param %s: %r' % (param, val))
        # setting xgboost param
        logger.info('Setting best parameters for BayesianOptimization')
        xgb_params = {
#             'tree_method':'gpu_hist',
#             'predictor':'gpu_predictor',
            'nthread': 12,
#             'n_estimators': int(750.6),
#             'eta': 0.01,
            'silent': 1,
            # for _train_internal
#             'eval_metric': ['map'],
            ######################
            'max_delta_step':int(best_params['max_delta_step']),
            'reg_lambda': max(best_params['reg_lambda'], 0),
            'reg_alpha': max(best_params['reg_alpha'], 0),
            'n_estimators':int(best_params['n_estimators']),
            'max_depth': int(best_params['max_depth']),
            'subsample': max(min(best_params['subsample'], 1), 0),
            'min_child_weight': int(best_params['min_child_weight']),
##             'alpha': max(best_params['alpha'], 0),
            'gamma': max(best_params['gamma'], 0),
            'colsample_bytree': max(min(best_params['colsample_bytree'], 1), 0),
            'colsample_bylevel': max(min(best_params['colsample_bylevel'], 1), 0),
#             'learning_rate': max(min(best_params['learning_rate'], 1), 0),
        }
        # training
        model = xgb.train(
            xgb_params, 
            dTrain, 
            num_boost_round=1500,  #2000
            verbose_eval=False, 
            maximize=False,
            
        )
        #save XGB best model
#         writePickle(model, os.path.join('model', 'model2_iter%d_%dfold_%f.pkl' % (iter, 3, xgb_bayesopt.res['max']['max_val'])))
        #n_fold
        # predict valid y
#         predY = model.predict(dTest)
#         result_df = pd.DataFrame(data={'y':predY})
#         joined_df = pd.DataFrame(testFid).join(result_df)


#         joined_df.to_csv(os.path.join('result', 'xgb_result%d_%dfold.csv' % (iter, 10)), index=False)


        

        # re-sorted the fid because of the random splitting data
        logger.info('----------------------------------------------------------------------\n\n\n')

In [ ]:
path = getPath()
dTrain = []
dTest = []
logger = Logger().getLogger()
train()

2018-06-15 22:03:54,433 - logHandler - train - INFO - Iteration  0, Current random seed:  0
2018-06-15 22:03:54,434 - logHandler - train - INFO - Setting parameters for BayesianOptimaization
2018-06-15 22:03:54,434 - logHandler - train - INFO - Running BayesianOptimization
Initialization
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bylevel |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   n_estimators |   reg_alpha |   reg_lambda |   subsample | 
    1 | 00m02s |   -0.15310 |              0.9050 |             0.9756 |    0.6188 |           9.2569 |      1.0000 |             9.7689 |      1134.4254 |      0.2315 |      77.8663 |      0.9381 | 
    2 | 00m02s |   -0.16145 |              0.9523 |             0.9167 |    0.8470 |           2.8036 |      1.0

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([6.41807405e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 62, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


   10 | 00m15s |   -0.16226 |              0.9507 |             0.8967 |    0.0181 |           9.9059 |      1.0000 |             8.4246 |      1134.7307 |      0.2208 |      84.8202 |      0.9345 | 
   11 | 00m13s |   -0.15580 |              0.9288 |             0.8084 |    0.9871 |           9.5536 |      1.0000 |             9.0318 |      1132.3789 |      0.1080 |      70.0325 |      0.9377 | 
   12 | 00m15s |   -0.15364 |              0.9884 |             0.9440 |    0.0116 |           9.2916 |      1.0000 |             8.1427 |      1129.4976 |      0.2449 |      75.1794 |      0.9667 | 
   13 | 00m15s |   -0.15579 |              0.9090 |             0.9923 |    0.0260 |           5.8841 |      1.0000 |            10.9649 |      1120.1556 |      0.0176 |      84.5917 |      0.9217 | 
   14 | 00m17s |   -0.16145 |              0.9279 |             0.8130 |    0.0866 |           0.5991 |      1.0000 |            10.9573 |      1134.2608 |      0.0436 |      84.9109 |      0.9804 | 


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([6.9040687e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   31 | 00m21s |   -0.15499 |              0.9106 |             0.8389 |    0.3637 |           0.5482 |      1.0000 |             8.2967 |      1129.8945 |      0.2880 |      84.8842 |      0.9927 | 


...

   59 | 00m44s |   -0.15741 |              0.9221 |             0.9891 |    0.0201 |           9.3286 |      1.0000 |             8.1496 |      1132.2340 |      0.2604 |      71.2338 |      0.9031 | 
   60 | 00m45s |   -0.15876 |              0.9021 |             0.8226 |    0.0610 |           1.8412 |      1.0000 |            10.7988 |      1134.8827 |      0.0151 |      78.1446 |      0.9159 | 
   61 | 00m42s |   -0.16361 |              0.9546 |             0.9669 |    0.7229 |           4.9170 |      1.0000 |            10.3001 |      1120.2181 |      0.2772 |      84.9640 |      0.9481 | 
   62 | 00m45s |   -0.15741 |              0.9018 |             0.9108 |    0.9792 |           4.4124 |      1.0000 |             8.3411 |      1134.9104 |      0.2847 |      70.0445 |      0.9456 | 
   63 | 00m43s |   -0.15984 |              0.9226 |             0.8342 |    0.0351 |           6.7791 |      1.0000 |             8.2501 |      1120.0807 |      0.0466 |      80.5303 |      0.9897 | 


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.0002353]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00035622]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


   65 | 00m47s |   -0.15661 |              0.9075 |             0.9061 |    0.0081 |           9.8031 |      1.0000 |             9.1472 |      1127.6574 |      0.2999 |      84.9855 |      0.9422 | 
   66 | 00m48s |   -0.15525 |              0.9510 |             0.8735 |    0.9806 |           1.9615 |      1.0000 |             8.1109 |      1134.8545 |      0.0328 |      76.9648 |      0.9940 | 
   67 | 00m49s |   -0.15580 |              0.9844 |             0.9171 |    0.1695 |           0.1606 |      1.0000 |            10.7819 |      1120.3304 |      0.0555 |      84.6961 |      0.9720 | 
   68 | 00m49s |   -0.15822 |              0.9289 |             0.8347 |    0.0291 |           2.8611 |      1.0000 |             8.6381 |      1134.9543 |      0.2129 |      73.1749 |      0.9750 | 
   69 | 00m45s |   -0.15903 |              0.9120 |             0.9606 |    0.9880 |           0.1067 |      1.0000 |            10.0501 |      1132.8140 |      0.0819 |      84.9647 |      0.9111 | 


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00033082]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


   73 | 00m52s |   -0.15660 |              0.9218 |             0.8127 |    0.0388 |           4.2622 |      1.0000 |            10.5902 |      1127.8278 |      0.0230 |      70.0055 |      0.9917 | 
   74 | 00m53s |   -0.16280 |              0.9403 |             0.9861 |    0.2112 |           6.7603 |      1.0000 |            10.9123 |      1123.1370 |      0.0209 |      76.9868 |      0.9654 | 
   75 | 00m47s |   -0.15391 |              0.9131 |             0.9040 |    0.7431 |           9.1974 |      1.0000 |            10.9470 |      1130.7768 |      0.2746 |      84.7922 |      0.9833 | 
   76 | 00m50s |   -0.15903 |              0.9001 |             0.8172 |    0.9927 |           0.0094 |      1.0000 |            10.4504 |      1120.5046 |      0.0244 |      70.0854 |      0.9830 | 
   77 | 00m54s |   -0.16092 |              0.9338 |             0.8676 |    0.9319 |           9.8418 |      1.0000 |            10.1881 |      1134.9184 |      0.1671 |      84.6758 |      0.9881 | 


## Model

In [111]:
sum(np.argmax(X_train,axis=1) == Y_train) / len(Y_train)

0.8194070080862533

In [112]:
xgb1 = xgb.XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor',objective='multi:softmax')
xgb1.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob',
       predictor='gpu_predictor', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1, tree_method='gpu_hist')

In [114]:
# xgb1.score(X_valid,Y_valid)
xgb1.score(X_train,Y_train)

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9982030548068284

## Transformer

In [136]:
len(X_train)

3339

In [118]:
one_hot_label = to_categorical(Y_train,num_classes=41)
one_hot_label_V = to_categorical(Y_valid,num_classes=41)

In [165]:
input_ = Input(shape=(41,))
bn = BatchNormalization()(input_)
dense = Dense(82,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.005))(bn)
# dense = Dense(82,kernel_initializer='uniform',kernel_regularizer=l2(0.008))(bn)
# dense = LeakyReLU()(dense)
dropout = Dropout(0.6)(dense)

# dense = Dense(164,kernel_initializer='uniform',kernel_regularizer=l2(0.008))(dropout)
bn = BatchNormalization()(dropout)
dense = Dense(82,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.005))(bn)
# dense = LeakyReLU()(dense)
dropout = Dropout(0.6)(dense)

bn = BatchNormalization()(dropout)
dense = Dense(41,activation='softmax',kernel_regularizer=l2(0.006),kernel_initializer='lecun_normal')(bn)
# dense = Dense(41,activation='softmax',kernel_regularizer=l2(0.001))(bn)
model = Model(inputs=input_, outputs=dense)
# model = Sequential()
# i
# model.add(BatchNormalization())
# model.add(Dense(41,activation='linear',input_shape=(41,)))


In [166]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(X_train,one_hot_label,validation_data=(X_valid,one_hot_label_V),epochs=5000,batch_size=1024)

Train on 3339 samples, validate on 371 samples
Epoch 1/5000
3339/3339 [==============================] - 1s 299us/step - loss: 4.9256 - acc: 0.0270 - val_loss: 4.3500 - val_acc: 0.1375
Epoch 2/5000
3339/3339 [==============================] - 0s 8us/step - loss: 4.5515 - acc: 0.0830 - val_loss: 3.9664 - val_acc: 0.4178
Epoch 3/5000
3339/3339 [==============================] - 0s 8us/step - loss: 4.2301 - acc: 0.1518 - val_loss: 3.5931 - val_acc: 0.6361
Epoch 4/5000
3339/3339 [==============================] - 0s 8us/step - loss: 3.8757 - acc: 0.2609 - val_loss: 3.2482 - val_acc: 0.7790
Epoch 5/5000
3339/3339 [==============================] - 0s 8us/step - loss: 3.5712 - acc: 0.3609 - val_loss: 2.9389 - val_acc: 0.8167
Epoch 6/5000
3339/3339 [==============================] - 0s 8us/step - loss: 3.2867 - acc: 0.4570 - val_loss: 2.6777 - val_acc: 0.8248
Epoch 7/5000
3339/3339 [==============================] - 0s 7us/step - loss: 3.0528 - acc: 0.5190 - val_loss: 2.4622 - val_acc: 0.8248

Epoch 120/5000
3339/3339 [==============================] - 0s 8us/step - loss: 1.0913 - acc: 0.8080 - val_loss: 0.9605 - val_acc: 0.8275
Epoch 121/5000
3339/3339 [==============================] - 0s 8us/step - loss: 1.0902 - acc: 0.8059 - val_loss: 0.9660 - val_acc: 0.8167
Epoch 122/5000
3339/3339 [==============================] - 0s 8us/step - loss: 1.0954 - acc: 0.8065 - val_loss: 0.9588 - val_acc: 0.8275
Epoch 123/5000
3339/3339 [==============================] - 0s 7us/step - loss: 1.0897 - acc: 0.8092 - val_loss: 0.9547 - val_acc: 0.8302
Epoch 124/5000
3339/3339 [==============================] - 0s 7us/step - loss: 1.0842 - acc: 0.8074 - val_loss: 0.9604 - val_acc: 0.8275
Epoch 125/5000
3339/3339 [==============================] - 0s 8us/step - loss: 1.0779 - acc: 0.8038 - val_loss: 0.9548 - val_acc: 0.8248
Epoch 126/5000
3339/3339 [==============================] - 0s 8us/step - loss: 1.0735 - acc: 0.8008 - val_loss: 0.9574 - val_acc: 0.8248
Epoch 127/5000
3339/3339 [========

3339/3339 [==============================] - 0s 7us/step - loss: 0.9812 - acc: 0.8104 - val_loss: 0.8593 - val_acc: 0.8302
Epoch 239/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.9702 - acc: 0.8179 - val_loss: 0.8574 - val_acc: 0.8302
Epoch 240/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.9814 - acc: 0.8122 - val_loss: 0.8520 - val_acc: 0.8275
Epoch 241/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.9842 - acc: 0.8080 - val_loss: 0.8447 - val_acc: 0.8329
Epoch 242/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.9613 - acc: 0.8158 - val_loss: 0.8441 - val_acc: 0.8356
Epoch 243/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.9687 - acc: 0.8113 - val_loss: 0.8495 - val_acc: 0.8302
Epoch 244/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.9851 - acc: 0.8086 - val_loss: 0.8551 - val_acc: 0.8221
Epoch 245/5000
3339/3339 [=======================

3339/3339 [==============================] - 0s 7us/step - loss: 0.9304 - acc: 0.8137 - val_loss: 0.8240 - val_acc: 0.8302
Epoch 357/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.9178 - acc: 0.8191 - val_loss: 0.8314 - val_acc: 0.8275
Epoch 358/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.9249 - acc: 0.8149 - val_loss: 0.8271 - val_acc: 0.8275
Epoch 359/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.9187 - acc: 0.8176 - val_loss: 0.8297 - val_acc: 0.8383
Epoch 360/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.9212 - acc: 0.8098 - val_loss: 0.8379 - val_acc: 0.8248
Epoch 361/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.9129 - acc: 0.8188 - val_loss: 0.8319 - val_acc: 0.8329
Epoch 362/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.9244 - acc: 0.8092 - val_loss: 0.8317 - val_acc: 0.8329
Epoch 363/5000
3339/3339 [=======================

3339/3339 [==============================] - 0s 7us/step - loss: 0.8901 - acc: 0.8110 - val_loss: 0.8326 - val_acc: 0.8194
Epoch 475/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.9002 - acc: 0.8161 - val_loss: 0.8254 - val_acc: 0.8248
Epoch 476/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.8915 - acc: 0.8095 - val_loss: 0.8276 - val_acc: 0.8194
Epoch 477/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.8914 - acc: 0.8155 - val_loss: 0.8232 - val_acc: 0.8194
Epoch 478/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.8927 - acc: 0.8080 - val_loss: 0.8262 - val_acc: 0.8248
Epoch 479/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.8944 - acc: 0.8092 - val_loss: 0.8164 - val_acc: 0.8275
Epoch 480/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.8796 - acc: 0.8122 - val_loss: 0.8241 - val_acc: 0.8140
Epoch 481/5000
3339/3339 [=======================

...

3339/3339 [==============================] - 0s 8us/step - loss: 0.7900 - acc: 0.8191 - val_loss: 0.7769 - val_acc: 0.8248
Epoch 4723/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7798 - acc: 0.8164 - val_loss: 0.7802 - val_acc: 0.8248
Epoch 4724/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7879 - acc: 0.8143 - val_loss: 0.7811 - val_acc: 0.8275
Epoch 4725/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7887 - acc: 0.8179 - val_loss: 0.7819 - val_acc: 0.8302
Epoch 4726/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7822 - acc: 0.8158 - val_loss: 0.7819 - val_acc: 0.8248
Epoch 4727/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7734 - acc: 0.8170 - val_loss: 0.7832 - val_acc: 0.8275
Epoch 4728/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7787 - acc: 0.8173 - val_loss: 0.7900 - val_acc: 0.8221
Epoch 4729/5000
3339/3339 [================

3339/3339 [==============================] - 0s 8us/step - loss: 0.7978 - acc: 0.8149 - val_loss: 0.8006 - val_acc: 0.8302
Epoch 4782/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7934 - acc: 0.8158 - val_loss: 0.8009 - val_acc: 0.8248
Epoch 4783/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7893 - acc: 0.8227 - val_loss: 0.7990 - val_acc: 0.8167
Epoch 4784/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7774 - acc: 0.8149 - val_loss: 0.7971 - val_acc: 0.8248
Epoch 4785/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7780 - acc: 0.8128 - val_loss: 0.7952 - val_acc: 0.8275
Epoch 4786/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7821 - acc: 0.8212 - val_loss: 0.7903 - val_acc: 0.8275
Epoch 4787/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7830 - acc: 0.8164 - val_loss: 0.7858 - val_acc: 0.8275
Epoch 4788/5000
3339/3339 [================

3339/3339 [==============================] - 0s 8us/step - loss: 0.7763 - acc: 0.8134 - val_loss: 0.7826 - val_acc: 0.8302
Epoch 4841/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.7882 - acc: 0.8197 - val_loss: 0.7854 - val_acc: 0.8302
Epoch 4842/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7692 - acc: 0.8227 - val_loss: 0.7829 - val_acc: 0.8302
Epoch 4843/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7849 - acc: 0.8155 - val_loss: 0.7796 - val_acc: 0.8275
Epoch 4844/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7983 - acc: 0.8131 - val_loss: 0.7721 - val_acc: 0.8356
Epoch 4845/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7815 - acc: 0.8233 - val_loss: 0.7724 - val_acc: 0.8302
Epoch 4846/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7876 - acc: 0.8203 - val_loss: 0.7746 - val_acc: 0.8248
Epoch 4847/5000
3339/3339 [================

3339/3339 [==============================] - 0s 8us/step - loss: 0.7742 - acc: 0.8194 - val_loss: 0.7928 - val_acc: 0.8221
Epoch 4900/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7887 - acc: 0.8173 - val_loss: 0.7868 - val_acc: 0.8221
Epoch 4901/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7812 - acc: 0.8185 - val_loss: 0.7839 - val_acc: 0.8302
Epoch 4902/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7893 - acc: 0.8185 - val_loss: 0.7865 - val_acc: 0.8302
Epoch 4903/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7726 - acc: 0.8170 - val_loss: 0.7848 - val_acc: 0.8248
Epoch 4904/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7917 - acc: 0.8113 - val_loss: 0.7816 - val_acc: 0.8248
Epoch 4905/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.8034 - acc: 0.8131 - val_loss: 0.7770 - val_acc: 0.8275
Epoch 4906/5000
3339/3339 [================

3339/3339 [==============================] - 0s 8us/step - loss: 0.7793 - acc: 0.8125 - val_loss: 0.7796 - val_acc: 0.8248
Epoch 4959/5000
3339/3339 [==============================] - 0s 7us/step - loss: 0.7781 - acc: 0.8152 - val_loss: 0.7802 - val_acc: 0.8275
Epoch 4960/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7899 - acc: 0.8104 - val_loss: 0.7882 - val_acc: 0.8248
Epoch 4961/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7932 - acc: 0.8116 - val_loss: 0.7944 - val_acc: 0.8194
Epoch 4962/5000
3339/3339 [==============================] - 0s 9us/step - loss: 0.8008 - acc: 0.8131 - val_loss: 0.7931 - val_acc: 0.8302
Epoch 4963/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7866 - acc: 0.8158 - val_loss: 0.7907 - val_acc: 0.8248
Epoch 4964/5000
3339/3339 [==============================] - 0s 8us/step - loss: 0.7789 - acc: 0.8185 - val_loss: 0.7840 - val_acc: 0.8275
Epoch 4965/5000
3339/3339 [================

NameError: name 'model' is not defined

In [157]:
ans = model.predict(X_valid)
ans = np.argmax(ans,axis=1)
ans

array([39,  0, 39, 25, 40,  9, 39,  6, 20,  0,  5, 15, 10, 13, 20, 32,  4,
       12,  2, 35, 29, 33,  4,  1,  1, 25,  7, 33, 20, 13,  7, 25,  7, 40,
        7, 20, 28, 29,  4,  6,  0, 37, 39,  1, 15,  8,  8, 31,  6, 28,  9,
       19, 13, 38, 17, 30,  2,  0, 39,  9,  2, 16, 32, 19, 39, 38, 40, 10,
       20,  8, 26, 31, 39,  4,  7, 27,  6,  9, 37, 18, 14, 19, 31,  6,  4,
       29,  9, 28,  5,  2, 25, 16, 12, 25,  4,  1, 40, 15, 32, 39, 16,  5,
        4, 29, 39, 35, 30, 16, 16, 33, 13, 12, 39,  8, 17, 17, 19,  2,  1,
       15, 39, 38,  4, 29, 32, 34, 12,  4, 12, 34, 39, 21, 14, 13,  6, 33,
       19, 15, 10, 27, 30, 32, 16, 39, 14, 34, 17,  0, 11,  4,  5, 28,  1,
       16, 38, 13, 34, 11, 18, 38, 39, 27, 24, 22, 18, 31, 39,  6,  7, 34,
       29, 32, 15, 10, 30,  0, 26, 10, 15,  8, 15, 27,  4, 33, 34, 24, 14,
        1, 13, 20, 16,  9,  1, 33, 38,  6, 22,  6,  6, 26, 14, 12, 20, 25,
       39,  8, 15, 11, 12,  7, 31, 28,  1, 20, 19, 11,  6, 14, 20, 39, 15,
       34,  4, 20, 10, 31

In [158]:
sum(ans == Y_valid) / len(Y_valid)

0.8274932614555256